In [8]:
import pandas as pd

In [9]:
npr = pd.read_csv('./writeups_attack_execution.csv')

In [10]:
# Check the first 5 rows of the csv
npr.head()

,Writeup_name,Category,Text,Label
0,XMarkTheSpot,web,"For example, let's inject:\nThis should tell u...",attack_execution
1,Members,web,We tried out some SQLi payloads to see that a ...,attack_execution
2,05_Cursed_Secret_Party,web,Then convert the link of payload file on githu...,attack_execution
3,High_Security,web,We then tried to change our IP address with th...,attack_execution
4,27_Forbidden_Paths,web,Since we know that we are in /usr/share/nginx/...,attack_execution


In [11]:
# Let's see the first text
npr['Text'][0]

'For example, let\'s inject:\nThis should tell us if the password of the first username is longer than 1 character. \nIf we submit this as the username, we get back a response from the server telling us that "You\'re on the right path", so we can deduce that password is longer than 1. \nIf we try this, on the other hand, we receive a "Login Failure": so the password is shorter than 30 characters.\nLet\'s use the following syntax to iterate some users and check if someone\'s password starts with "pico":\nWe\'ll use curl.\nWe got a match for the third user. Now let\'s extract his password with the following script.\nThe script uses XPATH substring to brute force the password character by character. As output we have the flag.'

In [12]:
# Number of rows of the dataset
len(npr)

52

In [13]:
# Preprocessing

from sklearn.feature_extraction.text import CountVectorizer

# max_df = ignore high frequency terms (0-1)
# min_df = ignore low frequency terms (min # of documents containing it)
cv = CountVectorizer(max_df=0.9, min_df=4, stop_words='english')

dtm = cv.fit_transform(npr['Text'])

dtm

<52x77 sparse matrix of type '<class 'numpy.int64'>'
	with 493 stored elements in Compressed Sparse Row format>

In [14]:
# LDA

from sklearn.decomposition import LatentDirichletAllocation

# n_components is the number of topics we are looking for
LDA = LatentDirichletAllocation(n_components=1, random_state=42)

LDA.fit(dtm)

LatentDirichletAllocation(n_components=1, random_state=42)

In [15]:
# Grab the vocabulary of words

import random

#len(cv.get_feature_names()) #589

random_word_id = random.randint(0,77)

cv.get_feature_names()[random_word_id]

/Users/marco/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


'username'

In [16]:
# Grab the topics

#len(LDA.components_) #3

#type(LDA.components_) #numpy.ndarray - Contains the words and their probability

# LDA.components_.shape # (3,589)

single_topic = LDA.components_[0]

single_topic.argsort() # returns an array containing the index position of each word instead of the word itself

# ARGSORT ----> INDEX POSITION SORTED FROM LEAST TO GREATEST
# TOP 10 VALUES (10 GREATEST VALUES)
# LAST 10 VALUES of argsort()
single_topic.argsort()[-10:] # grab the last 10 values of .argsort()

top_ten_words = single_topic.argsort()[-10:]

for index in top_ten_words:
    print(cv.get_feature_names()[index])

script
password
use
file
following
admin
url
request
payload
flag


In [18]:
# Grab the highest probability words per topic

for i,topic in enumerate(LDA.components_):
    print(f"THE TOP 15 WORDS FOR TOPIC #{i}")
    print([cv.get_feature_names()[index] for index in topic.argsort()[-100000:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['data', 'input', 'new', 'method', 'local', 'decode', 'js', 'finally', 'force', 'app', 'set', 'need', 'll', 'function', 'execute', 'javascript', 'got', 'line', 'python', 'read', 'brute', 'add', 'path', 'query', 'second', 'list', 'session', 'simple', 'txt', 'inject', 'website', 'html', 'character', 'check', 'final', 'create', 'php', 'challenge', 'change', 'parameter', 'filter', 'look', 'result', 'like', 'object', 'injection', 'bypass', 'username', 'source', 'cookie', 'access', 'gives', 'http', 'try', 'error', 'array', 'server', 'let', 'tried', 'just', 'login', 'user', 'code', 'using', 'page', 'string', 'receive', 'script', 'password', 'use', 'file', 'following', 'admin', 'url', 'request', 'payload', 'flag']




In [63]:

# Put in column the top
topic_results = LDA.transform(dtm)

npr['Topic'] = topic_results.argmax(axis=1)

topic_dictionary = {0:'Topic 0',1:'Topic 1',2:'Topic 2'}

npr['Topic Label'] = npr['Topic'].map(topic_dictionary)

In [64]:
type(npr)

npr.to_csv(r'LDA_export_dataframe.csv', index=False, header=True)